

Entender fácilmente BeautifulSoup con html: https://unipython.com/web-scraping-con-beautiful-soup-iii/



Parte del código que creé como muestra para la formación en Samsung Desarrolladoras tras analizar el código de la Fundación Universia y acceder al JSON contenedor de sus becas.

In [1]:
import requests
import json
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime

#CAPTURA 1 - FUENTE - UNIVERSIA
resp = requests.get('https://api-manager.universia.net/agregador-scholarships/v1/api/scholarship?origin=ES&page=1')
data = json.loads(resp.content)
data2 = data['scholarships']
total_items = data['totalItems']    #
print('total items: ', total_items) #
df = pd.DataFrame(data2)

pages = data['totalPages']

#UNIÓN DE FRAGMENTOS
if (pages > 1 ):
  for i in range(2, (pages + 1)):
    res = requests.get('https://api-manager.universia.net/agregador-scholarships/v1/api/scholarship?origin=ES&page=' + str(i))
    data_res = json.loads(res.content)
    data3 = data_res['scholarships']
    data_res2 = pd.DataFrame(data_res['scholarships'])
    df = df.append(data_res2, ignore_index=True)
df

#LIMPIANDO DATOS

# Eliminamos columnas que no nos interesan
df.drop(['countryCode', 'callDate', 'knowledgeAreas', 'applicantLocation', 'destinationLocation', 'elegibleRegionAll', 'areaServedAll', 'suggest'], axis = 1, inplace = True)

# Sustiyendo valores NaN en precio
df['price'].fillna(0, inplace = True)
df['price'] =[int(e) for e in df['price'] ]

pr = ['gratis' if p==0 else str(p) for p in df['price']]
df['price'] = pr

# Modificando el objeto 'summoner' (convocante) para obtener 'name y 'url' en un string
summoner= df['summoner']
summoner = [str(e['name']) + ' ' for e in summoner]
df['summoner'] = summoner 

#Columna 'schorlashipTypes' = Tipo = Becas + tipo de beca
#Los códigos numéricos que aparecen en el json se corresponden con el tipo de beca.
# Eso, obviamente, lo he tenido que sacar a mano mirando la web y comparando con el JSON

tipos = df['scholarshipTypes']
t2 = [] 
for tipo in tipos:
  t = "" 
  for e in tipo:     
    if e == 17:
      e = 'Becas. Premios, concursos, certámenes. '
    if e == 14:
      e = 'Beca de idiomas. '
    if e == 13:
      e = 'Beca de formación de profesorado. '
    if e == 12:
      e = 'Beca de estudios. '
    if e == 10:
      e = 'Beca de creación artística. '
    if e == 7:
      e = 'Beca de movilidad. '
    if e == 6:
      e = 'Beca de investigación. ' 
    if e == 5:
      e = 'Beca de formación práctica. '
    if e == 2:
      e = 'Asistencia a cursos, conferencias y congresos. '
    if e == 1:
      e = 'Beca de alojamiento, manutención y transporte. '
    t += e 
    t     
  t2.append(t)
df['scholarshipTypes'] = t2

# userProfiles de código numérico a etiqueta, los reducimos a la etiqueta más básica

perfil = df.userProfiles

perfiles = []
for user in perfil:
  u = ""
  for f in user:
    if f == 339:
      f = 'postuniversitarios '
    if f == 338:
      f = 'universitarios '
    if f == 337:
      f = 'preuniversitarios '
    u += f
  perfiles.append(u)
df.userProfiles = perfiles  

# Limpiando el texto de la descripción... 'BeautifulSoup'
description = df['description']
description_2 = []

for x in description:
  doc = str(x) 
  soup = BeautifulSoup(doc, 'html.parser')
  description_2.append(str(soup.get_text()))

df['description'] = description_2

# Formateando fechas para que queden bonitas :P

def fechas(fecha):
  d = pd.to_datetime(fecha, infer_datetime_format=True, errors='ignore')
  d_format = [e.strftime('%d - %m - %Y') for e in d]
  fecha = d_format
  return fecha

df.endDate = fechas(df['endDate'])
df.startDate = fechas(df['startDate'])

#Añadiendo culumnas al df para el enum de Bixby: Becas & ayudas, para citar la fuente
# de extracción y para añadir la columna de fecha de publicación, que usaremos con los 
# datos de las ayudas del BOE

t_beca = ['becas' for b in df['title']]
df.insert(2,'tipoConvocatoria', t_beca, True)

f_beca = ['Universia' for c in range(len(df['title']))]
df.insert(1,'fuente', f_beca, True)

#Generando archivos para vista interna y para la creación del JSON que nos exportará 
# cada vez que corramos python. La opción orient = 'records' nos permite mantener
# la estructura json con los valores dispuestos como deseaba.

becas = df.to_json(orient='records', force_ascii=True)

# Esta concatenación de strings es para que nuestro JSON funcione como un módulo JS
# que es lo que me interesaba para este proceso. 

becas2 = str('module.exports = ' + becas)

with open("becas.js", "w") as file:
  if file: 
    file.truncate(0)
  file.write(becas2)
  file.close()

print('Visión de nuestros datos actuales en DataFrame: \n')
df


total items:  122
Visión de nuestros datos actuales en DataFrame: 



,title,fuente,description,tipoConvocatoria,id,price,userProfiles,url,startDate,summoner,scholarshipTypes,endDate
0,Becas Fulbright para estudios de máster o doct...,Universia,Destinadas a jóvenes titulados superiores que ...,becas,241472,gratis,postuniversitarios,https://fulbright.es/programas-y-becas/convoca...,07 - 04 - 2020,Comisión Fulbright España,Beca de movilidad. Beca de estudios.,23 - 04 - 2020
1,Becas para mexicanos para estudiar en Australia,Universia,"La Secretaría de Educación Pública (SEP), a tr...",becas,261512,150000,postuniversitarios,https://www.dgri.sep.gob.mx/formatos/macquarie...,17 - 03 - 2020,Secretaria de Educación Publica,Beca de estudios.,24 - 04 - 2020
2,Becas para estudiar el doctorado de Ciencias E...,Universia,El Banco de la República de Colombia ofrece ci...,becas,261522,gratis,postuniversitarios,https://www.banrep.gov.co/es/doctorado-economia,17 - 03 - 2020,Banco de la República de Colombia,Beca de estudios.,24 - 04 - 2020
3,"Becas de doctorado para estudiar en Londres, C...",Universia,La prestigiosa Escuela de Ciencias Políticas y...,becas,258437,18000,postuniversitarios,https://www.lse.ac.uk/study-at-lse/Graduate/Fe...,20 - 11 - 2019,London School of Economics (LSE),Beca de estudios.,27 - 04 - 2020
4,Becas MAEC-AECID,Universia,La Agencia Española de Cooperación Internacion...,becas,260050,20000,postuniversitarios,http://www.aecid.es/ES/becas-y-lectorados/conv...,13 - 01 - 2020,Ministerio de Asuntos Exteriores y AECID,Beca de estudios.,27 - 04 - 2020
...,...,...,...,...,...,...,...,...,...,...,...,...
117,Programa de Becas del Gobierno Chino,Universia,El Programa Gran Muralla es una beca completa ...,becas,261687,gratis,postuniversitarios,https://apply.china-admissions.com/search/,31 - 12 - 2020,Gobierno de China,Beca de estudios.,31 - 03 - 2021
118,Becas STEM de Intertech,Universia,Uno de los enfoques de la Fundación Intertech ...,becas,261608,2500,universitarios,https://www.intertech.com/about/foundation/sch...,01 - 01 - 2021,Intertech,Beca de estudios.,10 - 05 - 2021
119,Beca AICPA para estudiantes de doctorado de mi...,Universia,La beca AICPA para estudiantes de doctorado de...,becas,261694,gratis,postuniversitarios,https://www.aicpa.org/career/diversityinitiati...,01 - 03 - 2021,(FIA) Festival Internacional Audiovisual,Beca de estudios.,29 - 05 - 2021
120,Becas de Formación FUNIBER,Universia,La Fundación Universitaria Iberoamericana (FUN...,becas,261678,gratis,universitarios postuniversitarios,https://www.funiber.org/becas-maestria,30 - 03 - 2020,Fundación Universitaria Iberoamericana (Funiber),Beca de estudios.,30 - 12 - 2021
